In [ ]:
import numpy as np
import re
import unicodedata
from gensim.models import Word2Vec
import spacy
import nltk
import pandas as pd
from contractions import CONTRACTION_MAP #contractions.py file must be in the same directory

## Load Spacy Language Modules

In [23]:
#nlp=spacy.load('en_core_web_lg')
nlp = spacy.load('en_vectors_web_lg')
#en_vectors_web_lg.load()
total_vectors = len(nlp.vocab.vectors)
total_vectors #1070971

1070971

In [3]:
train=pd.read_csv('train.csv')
#test=pd.read_csv('test.csv')

In [ ]:
short_train=train[:10000]

In [ ]:
train=short_train

## Text Pre-processsing Functions

In [13]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [14]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text



In [15]:
def remove_special_characters(text):
    text = re.sub(r'[\,]', ', ', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    #text = re.sub("[^a-zA-z0-9'\s]", '', txt)
    return text

In [16]:
def lowercase(doc1):
    doc1=nlp(doc1)
    no_upper=' '.join([token.text.lower() for token in doc1 ])
    return no_upper

In [17]:
def tokenize_lemmatize(comment, join=True):
    #comment=comment.lower()
    doc=nlp(comment)
    tokens = [token for token in (doc)]
    #tokens = [token.text.lower() for token in tokens]
    #tokens=' '.join(tokens)
    #lemmas = [token.lemma_ if ((token.pos_!='PRON') & (token.lemma_.isalpha()==True)) for token in tokens]
    #lemmas = [token.lemma_ for token in (tokens) if ((token.pos_!='PRON') & (token.lemma_.isalnum()==True)& (token.is_stop==False))]
    lemmas = [token.lemma_ for token in (tokens) if ((token.pos_!='PRON') & (token.lemma_.isalnum()==True))] #& (token.is_stop==False))]
    if join:
        lemmatized=' '.join(lemmas)
        return lemmatized
    else:
        return lemmas

In [18]:
def tokenize_lemmatize_stops(comment, join=True):
    #comment=comment.lower()
    doc=nlp(comment)
    tokens = [token for token in (doc)]
    #tokens = [token.text.lower() for token in tokens]
    #tokens=' '.join(tokens)
    #lemmas = [token.lemma_ if ((token.pos_!='PRON') & (token.lemma_.isalpha()==True)) for token in tokens]
    #lemmas = [token.lemma_ for token in (tokens) if ((token.pos_!='PRON') & (token.lemma_.isalnum()==True)& (token.is_stop==False))]
    lemmas = [token.lemma_ for token in (tokens) if ((token.pos_!='PRON') & (token.lemma_.isalnum()==True) & (token.is_stop==False))]
    if join:
        lemmatized=' '.join(lemmas)
        return lemmatized
    else:
        return lemmas

In [19]:
def remove_stops (doc):
    if type(doc) is str:
        doc=nlp(doc)
 
    return (' '.join([token.text for token in doc if token.is_stop==False]))

In [20]:
def remove_url (doc):
   
    return re.sub(r'http*\S+', '', doc) # remove URLs

In [21]:
def join_line(line):
    joined=' '.join(line)
    return joined

In [11]:
def transform_text(self):
    self=self.apply(lambda x: remove_url(x))
    self=self.apply(lambda x: remove_accented_chars(x))
    self=self.apply(lambda x: expand_contractions(x,CONTRACTION_MAP))
    #remove_accented_chars(text)
    self=self.apply(lambda x: remove_special_characters(x))
    #expand_contractions(text,CONTRACTION_MAP)
    self=self.apply(lambda x: lowercase(x))
    self=self.apply(lambda x: tokenize_lemmatize_stops(x,join=True))
    return self

In [13]:
comment_lemma=train.comment_text.apply(lambda x: remove_url(x))
comment_lemma=train.comment_text.apply(lambda x: remove_accented_chars(x))
comment_lemma=train.comment_text.apply(lambda x: expand_contractions(x,CONTRACTION_MAP))
#remove_accented_chars(text)
comment_lemma=train.comment_text.apply(lambda x: remove_special_characters(x))
#expand_contractions(text,CONTRACTION_MAP)


In [14]:
comment_lemma=comment_lemma.apply(lambda x: lowercase(x))
#comment_lemma=comment_lemma.apply(lambda x: remove_stops(x))


In [15]:
comment_lemma=train.comment_text.apply(lambda x: tokenize_lemmatize_stops(x,join=False))

In [16]:
train['comment_lemma']=comment_lemma

## Preliminary EDA

In [28]:
train[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].sum()*100/len(train)

toxic            9.584448
severe_toxic     0.999555
obscene          5.294822
threat           0.299553
insult           4.936361
identity_hate    0.880486
dtype: float64

In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 9 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
comment_lemma    159571 non-null object
dtypes: int64(6), object(3)
memory usage: 11.0+ MB


In [29]:
train[train.iloc[:,2:8].sum(axis='columns')>5].iloc[4:,1:9]

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_lemma
13964,I am going to murder ZimZalaBim ST47 for being...,1,1,1,1,1,1,"[go, murder, ZimZalaBim, ST47, evil, homosexua..."
22158,FUCK YOU!!!!!!!!!!!! YOU FUCKING NIGGER BAG OF...,1,1,1,1,1,1,"[FUCK, FUCKING, NIGGER, BAG, SHIT, HOPE, DIE, ..."
29968,u motherfukkin bitch i want to rape you smelly...,1,1,1,1,1,1,"[u, motherfukkin, bitch, want, rape, smelly, w..."
32098,Fuck All Asyriac Nation \n\nQamishli belong to...,1,1,1,1,1,1,"[Fuck, Asyriac, Nation, Qamishli, belong, Armi..."
33951,GO FUCK YOURSELF BITCH. I HATE YOUR SOULD. M...,1,1,1,1,1,1,"[FUCK, BITCH, HATE, SOULD, MOTHER, FUCKER, HEL..."
38513,AM GOING TO RAPE YOU IN THE ASS YOU FAT BITCH ...,1,1,1,1,1,1,"[GOING, RAPE, ASS, FAT, BITCH, WHEELCHAIRI, GO..."
38578,"fuck you honkey, why you hatin' on blacks? You...",1,1,1,1,1,1,"[fuck, honkey, hatin, black, fuck, pussy, geek..."
46155,"So fuck off and die, rape fantasising sluts",1,1,1,1,1,1,"[fuck, die, rape, fantasise, slut]"
55160,ANYONE WHO SUPPORTS THIS IS FUCKING SICK. MEN ...,1,1,1,1,1,1,"[SUPPORTS, FUCKING, SICK, MEN, UNDERAGED, FUCK..."
59279,JEW \n\nGet the fuck out of here you jewish so...,1,1,1,1,1,1,"[JEW, fuck, jewish, son, bitch, will, rape, fu..."


In [ ]:
comment_lemma.values

## Gensim Word2Vec

In [ ]:
import gensim

In [ ]:
model = gensim.models.Word2Vec(
        comment_lemma.values,
        size=150,
        window=10,
        min_count=4,
        workers=10)
model.train(comment_lemma.values, total_examples=len(comment_lemma.values), epochs=10)

In [ ]:
word1='desert'
model.wv.most_similar(positive=word1, topn=10)

In [ ]:
model.corpus_count

In [ ]:
similar_words = {search_term: [item[0] for item in model.wv.most_similar([search_term], topn=6)]
                  for search_term in ['nazi', 'good', 
                                      'mountain', 'america','red']}
similar_words

### TSNE and Similarities

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

l1=[[key]+value for key,value in similar_words.items()]
l1=sum(l1,[])
wvs = model.wv[l1]

tsne = TSNE(n_components=2, random_state=0, n_iter=10000, perplexity=2)
##np.set_printoptions(suppress=True)
T = tsne.fit_transform(wvs)
labels = l1

plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='red', edgecolors='r')
plt.title('word embeddings')
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+5, y+5), xytext=(0, 0), textcoords='offset points')
plt.show()

## Additional EDA and Word Cloud

In [97]:
from collections import Counter

In [ ]:
a=comment_lemma.values.sum()
corpus=(' '.join(a))
corpus=corpus.lower()
nlp.max_length=len(corpus)
nlp.Defaults.stop_words.add("wp")
corpus_nlp=nlp(corpus)


In [ ]:
bow=[token.text for token in corpus_nlp if not token.is_stop]

a=Counter(bow).most_common(50)
bb=' '.join(bow)

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
bb=' '.join(bow)

In [ ]:
import matplotlib.pyplot as plt
# Start with one review:
text = bb
stopwords_c = set(STOPWORDS)
stopwords_c.update(["article", "wikipedia", "edit", "talk", "balls"])
# Create and generate a word cloud image:
wordcloud = WordCloud().generate(bb)
wordcloud = WordCloud(stopwords=stopwords_c,max_font_size=50, max_words=100, background_color="white").generate(text)

plt.figure(figsize=(9,9))
plt.imshow(wordcloud, interpolation="bilinear")

plt.axis("off")
plt.show()

In [ ]:
train_full=pd.read_csv('train.csv')

In [ ]:

train_full[train_full.iloc[:,2:8].sum(axis='columns')>=6].count()
#train_full[train.iloc[:,2:8].sum(axis='columns')>=1]
train_full[train_full.iloc[:,2:8].sum(axis='columns')>=6].comment_text.values

In [ ]:
toxic=train_full.toxic.sum()
severe=train_full.severe_toxic.sum()
obscene=train_full.obscene.sum()
threat=train_full.threat.sum()
insult=train_full.insult.sum()
identity_hate=train_full.identity_hate.sum()

In [ ]:
train_full.columns

In [ ]:
comment_lemma2=comment_lemma.apply(lambda x: join_line(x))

In [ ]:
comment_lemma2

### Similarity Matrix

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tv_matrix=[]
similarity_df=[]
similarity_matrix=[]
len(train)

In [ ]:
tv = TfidfVectorizer(min_df=0.1, max_df=0.85, use_idf=True)
tv_matrix = tv.fit_transform(comment_lemma2)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)


#from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df


### Dendrogram

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
Z = linkage(similarity_matrix, 'ward')
pd.DataFrame(Z, columns=['Document\Cluster 1', 'Document\Cluster 2', 
                         'Distance', 'Cluster Size'], dtype='object')

In [ ]:
plt.figure(figsize=(8, 3))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Data point')
plt.ylabel('Distance')

dendrogram(Z, truncate_mode='lastp')
plt.xticks(rotation=90)
plt.axhline(y=275, c='gray', ls='--', lw=1.5)

### Multinomial Naive Bayes

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
y=train.toxic

In [ ]:
comment_lemma2=comment_lemma.apply(lambda x: join_line(x))

In [ ]:
counter = CountVectorizer(analyzer='word',ngram_range=(1, 3))

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
#counter.fit(list(xtrain) + list(xvalid))
xtrain_count =  counter.fit_transform(comment_lemma2) 

# Fitting a simple Logistic Regression on Counts
clf = MultinomialNB()
clf.fit(xtrain_count, y)
predictions = clf.predict_proba(xtrain_count)

In [ ]:
predictions

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y, predictions[:,1])

In [7]:
xtest=pd.read_csv('test.csv')

In [8]:
train=pd.read_csv('train.csv')

In [9]:
train_val=train[125000:]

In [10]:
train=train[:125000]

In [24]:
train_val['transformed']=transform_text(train_val.comment_text)

In [25]:
train['transformed']=transform_text(train.comment_text)

In [26]:
xtest['transformed']=transform_text(xtest.comment_text)

In [27]:
concat=pd.concat([train.transformed,train_val.transformed,xtest.transformed],axis='rows') 

## Class Weight

In [114]:
from sklearn.utils import class_weight
y=train.identity_hate
class_weight = (class_weight.compute_sample_weight('balanced',np.unique(y),y))
d = dict(enumerate(class_weight, 0))
sample_weight=y.map(d)

In [151]:
class_weight

array([ 0.50435765, 57.87037037])

In [1]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

### Count Vectorizer and MNB
### Identity Hate

In [144]:
y=train.identity_hate
y_val=train_val.identity_hate
counter = CountVectorizer(min_df=2,analyzer='word',ngram_range=(1, 3))


counter.fit(concat) 
train_count=counter.transform(train.transformed)
train_val_count=counter.transform(train_val.transformed)
test_count=counter.transform(xtest.transformed)

clf = MultinomialNB()
clf.fit(train_count, y, sample_weight=sample_weight)
predictions = clf.predict_proba(train_val_count)
accuracy_score(y_val, predictions[:,1].round())


0.9630325995776807

In [154]:
acc=np.round(accuracy_score(y_val, predictions[:,1].round()),4)
auc=np.round(roc_auc_score(y_val, predictions[:,1].round()),4)
prc=np.round(precision_score(y_val, predictions[:,1].round()),4)
rec=np.round(recall_score(y_val, predictions[:,1].round()),4)
print('Accuracy Score is {}, precision score is {},\n\
Recall score is {}, roc_auc score is {}'.format(acc,prc,rec,auc))

Accuracy Score is 0.963, precision score is 0.154,
Recall score is 0.6523, roc_auc score is 0.8091


In [128]:
test_pred=clf.predict_proba(test_count)[:,1].round().astype(int)
test_pred[test_pred==1].sum()

18330

In [129]:
pd.set_option('display.max_colwidth', 300)
xtest['identity_hate']=test_pred
xtest[xtest.identity_hate==1].comment_text

0         Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupa...
21                                                                                                                                                                                                         == Arabs are committing genocide in Iraq, but no protests in Europe. == \n\n May Europe also burn in hell.
28                                                                                               . \n\n           Jews are not a race because you can only get it from your mother. Your own mention of Ethiopian Jews not testing \n           as Jews proves it is not, as well as the fact that we accept converts
48                                                                    

In [135]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val, predictions[:,1].round())

array([[34382,    91],
       [   86,    12]], dtype=int64)

## TFIDF Vectorizer and Logistic Regression
### Toxic

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer

word_vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1, 4), max_features=20000)
word_vectorizer.fit(concat)
train_word_features = word_vectorizer.transform(train.transformed)
val_word_features = word_vectorizer.transform(train_val.transformed)
test_word_features = word_vectorizer.transform(xtest.transformed)


In [ ]:
y=train.toxic
y_val=train_val.toxic

In [64]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='sag')
classifier.fit(train_word_features, y)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [65]:
predictions = classifier.predict_proba(val_word_features)[:,1]
accuracy_score(y_val, predictions.round())

0.9563796245408

In [67]:
acc=np.round(accuracy_score(y_val, predictions.round()),4)

auc=np.round(roc_auc_score(y_val, predictions.round()),4)

prc=np.round(precision_score(y_val, predictions.round()),4)

rec=np.round(recall_score(y_val, predictions.round()),4)

print('Accuracy Score is {}, precision score is {},\n\
Recall score is {}, roc_auc score is {}'.format(acc,prc,rec,auc))


Accuracy Score is 0.9564, precision score is 0.9159,
Recall score is 0.5975, roc_auc score is 0.7959


## SVC with SVD and Standard Scaler
### Toxic 

In [73]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC


svd = TruncatedSVD(n_components=120)
svd.fit(train_word_features)
xtrain_svd = svd.transform(train_word_features)
xvalid_svd = svd.transform(val_word_features)


scl = StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)



In [ ]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, y)
predictions = clf.predict_proba(xvalid_svd_scl)[:,1]


In [78]:
acc=np.round(accuracy_score(y_val, predictions1.round()),4)

auc=np.round(roc_auc_score(y_val, predictions1.round()),4)

prc=np.round(precision_score(y_val, predictions1.round()),4)

rec=np.round(recall_score(y_val, predictions1.round()),4)

print('Accuracy Score is {}, precision score is {},\n\
Recall score is {}, roc_auc score is {}'.format(acc,prc,rec,auc))

Accuracy Score is 0.9463, precision score is 0.8909,
Recall score is 0.4977, roc_auc score is 0.7456


In [88]:
train_val['toxic_pred']=predictions1.round().astype(int)
train_val

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,transformed,toxic_pred
125000,9cb11411a3a94e19,"Incidentally, on p. 48 Alan says that another ...",0,0,0,0,0,0,incidentally p alan say juror brincknell inque...,0
125001,9cb1f44c3bebb5cc,All those stupid little blocks will not stop m...,1,0,0,0,0,0,stupid little block stop patient come,1
125002,9cb268a7585e61cf,Here it states Andre's measurements at death a...,0,0,0,0,0,0,state andres measurement death pound state and...,0
125003,9cb3c3eef1cca663,Wondering if you could... \n\nUse your oversig...,0,0,0,0,0,0,wonder use oversight power delete user page hi...,0
125004,9cb52a3e27b7cb04,Please stop \n\n Please stop. If you continue ...,0,0,0,0,0,0,stop stop continue vandalize page deliberately...,0
125005,9cb79e38264a9d3b,hmm i dont totally agree with you because why ...,0,0,0,0,0,0,hmm not totally agree vote vote not count vote...,0
125006,9cb98536cb38a798,"""\nAlso, when I click on the link to the artic...",0,0,0,0,0,0,click link article hindi article speak,0
125007,9cb9864600fd99ac,December 2012 (UTC)\nI can see it in this sect...,0,0,0,0,0,0,december utc section,0
125008,9cb9b69d84e39e2a,"""\n\nThen use . That's what it's for. ···? · T...",0,0,0,0,0,0,use talk nihonjoe,0
125009,9cbca3fcad283939,oscillates between indepth serious stuff and f...,0,0,0,0,0,0,oscillate indepth stuff frivolous celebrity en...,0


### Threat

In [89]:
y=train.threat
y_val=train_val.threat

In [90]:
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, y) # This time let's see the results for threat
predictions = clf.predict_proba(xvalid_svd_scl)

In [92]:
predictions2=predictions[:,1]

acc=np.round(accuracy_score(y_val, predictions2.round()),4)

auc=np.round(roc_auc_score(y_val, predictions2.round()),4)

prc=np.round(precision_score(y_val, predictions2.round()),4)

rec=np.round(recall_score(y_val, predictions2.round()),4)

print('Accuracy Score is {}, precision score is {},\n\
Recall score is {}, roc_auc score is {}'.format(acc,prc,rec,auc))

Accuracy Score is 0.9971, precision score is 0.3333,
Recall score is 0.0204, roc_auc score is 0.5101


## Logistic Regression
### Threat

In [93]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='sag')
classifier.fit(train_word_features, y)
predictions3 = classifier.predict_proba(val_word_features)[:,1]

In [94]:
predictions3=predictions[:,1]

acc=np.round(accuracy_score(y_val, predictions3.round()),4)

auc=np.round(roc_auc_score(y_val, predictions3.round()),4)

prc=np.round(precision_score(y_val, predictions3.round()),4)

rec=np.round(recall_score(y_val, predictions3.round()),4)

print('Accuracy Score is {}, precision score is {},\n\
Recall score is {}, roc_auc score is {}'.format(acc,prc,rec,auc))

Accuracy Score is 0.9971, precision score is 0.3333,
Recall score is 0.0204, roc_auc score is 0.5101


In [ ]:
from collections import Counter
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(comment_lemma)
document_count = Counter(X.indices).values()

plt.figure(figsize=(10,5))
plt.hist(document_count, bins = 10000, alpha = 0.75, cumulative = True, normed = 1)

plt.axis([0,11,0,1])
plt.xticks(range(0,100,5))

plt.xlabel("Document Count")
plt.ylabel("% of words that appear less than x times")
plt.show()
